In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
spark

In [3]:
orders_base = spark.sparkContext.textFile("/public/trendytech/retail_db/orders")

In [4]:
customers_base = spark.sparkContext.textFile("/public/trendytech/retail_db/customers")

In [5]:
orders_details_base = spark.sparkContext.textFile("/public/trendytech/retail_db/order_items")

In [6]:
order_items_map = orders_details_base.map(lambda x : ((int(x.split(",")[1]),float(x.split(",")[4]))))

In [7]:
orders_base_map = orders_base.map(lambda x:  (int(x.split(",")[0]),(int(x.split(",")[2]))))

In [8]:
join_rdd = order_items_map.join(orders_base_map)

In [9]:
mapped_rdd = join_rdd.map(lambda x : (x[1][1],x[1][0]))

In [10]:
reduced_rdd = mapped_rdd.reduceByKey(lambda x,y : x+y).sortBy(lambda x : x[1],False)

In [11]:
reduced_rdd.take(10)

[(791, 10524.169999999996),
 (9371, 9299.029999999999),
 (8766, 9296.14),
 (1657, 9223.710000000001),
 (2641, 9130.919999999998),
 (1288, 9019.11),
 (3710, 9019.099999999999),
 (4249, 8918.85),
 (5654, 8904.95),
 (5624, 8761.98)]

In [12]:
orders_details_base.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

In [13]:
mapped1_orders_details = orders_details_base.map(lambda x : (int(x.split(",")[2]), (int(x.split(",")[3]))))

In [14]:
mapped1_orders_details.take(5)

[(957, 1), (1073, 1), (502, 5), (403, 1), (897, 2)]

In [15]:
reduced1_rdd = mapped1_orders_details.reduceByKey(lambda x,y : x+y)

In [16]:
top_products = reduced1_rdd.sortBy(lambda x : x[1], False)

In [17]:
top_products.take(10)

[(365, 73698),
 (502, 62956),
 (1014, 57803),
 (191, 36680),
 (627, 31735),
 (403, 22246),
 (1004, 17325),
 (1073, 15500),
 (957, 13729),
 (977, 998)]

In [18]:
customers_base.take(5)

['1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521',
 '2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126',
 '3,Ann,Smith,XXXXXXXXX,XXXXXXXXX,3422 Blue Pioneer Bend,Caguas,PR,00725',
 '4,Mary,Jones,XXXXXXXXX,XXXXXXXXX,8324 Little Common,San Marcos,CA,92069',
 '5,Robert,Hudson,XXXXXXXXX,XXXXXXXXX,"10 Crystal River Mall ",Caguas,PR,00725']

In [19]:
mapped2_customers_base = customers_base.map(lambda x : (x.split(",")[6]))

In [20]:
filtered2_customers_base = mapped2_customers_base.filter(lambda x : x == 'Caguas')

In [21]:
filtered2_customers_base.count()

4584

In [22]:
mapped3_customers_base = customers_base.map(lambda x : (x.split(",")[7],1))

In [23]:
reduced3_customers_base = mapped3_customers_base.reduceByKey(lambda x,y : x+y).sortBy(lambda x : x[1], False)

In [24]:
reduced3_customers_base.take(3)

[('PR', 4771), ('CA', 2012), ('NY', 775)]

In [25]:
order4_items_map = orders_details_base.map(lambda x : ((int(x.split(",")[1]),float(x.split(",")[4]))))

In [26]:
orders4_base_map = orders_base.map(lambda x:  (int(x.split(",")[0]),(int(x.split(",")[2]))))

In [27]:
join4_rdd = order4_items_map.join(orders4_base_map)

In [28]:
mapped4_rdd = join4_rdd.map(lambda x : (x[1][1],x[1][0]))

In [29]:
reduced4_rdd = mapped4_rdd.reduceByKey(lambda x,y : x+y)

In [30]:
final4_rdd = reduced4_rdd.filter(lambda x : x[1]>1000).cache()

In [31]:
final4_rdd.count()

11148

In [37]:
orders5_base_map = orders_base.map(lambda x:  (int(x.split(",")[2]),(x.split(",")[3]))).filter(lambda x : (x[1]== 'CLOSED'))

In [38]:
customers4_base = customers_base.map(lambda x:  (int(x.split(",")[0]),(x.split(",")[7])))

In [39]:
join5_rdd = orders5_base_map.join(customers4_base)

In [43]:
mapped5_rdd = join5_rdd.map(lambda x : (x[1][1],1))

In [45]:
reduced5_result = mapped5_rdd.reduceByKey(lambda x,y : x+y).sortBy(lambda x : x[1], False)

In [46]:
reduced5_result.take(10)

[('PR', 2891),
 ('CA', 1232),
 ('NY', 450),
 ('TX', 403),
 ('IL', 313),
 ('FL', 220),
 ('OH', 172),
 ('MI', 167),
 ('PA', 157),
 ('NJ', 151)]

In [47]:
orders6_base_map = orders_base.map(lambda x:  (int(x.split(",")[2]),1))

In [50]:
reduced6_result = orders6_base_map.reduceByKey(lambda x,y : x+y).filter(lambda x : x[1]>=1)

In [52]:
reduced6_result.count()

12405

In [53]:
orders7_base_map = orders_base.map(lambda x:  (int(x.split(",")[0]),(int(x.split(",")[2]))))

In [54]:
mapped7_customers_base = customers_base.map(lambda x : ((int(x.split(",")[0])),x.split(",")[7]))

In [55]:
order7_items_map = orders_details_base.map(lambda x : ((int(x.split(",")[1]),float(x.split(",")[4]))))

In [56]:
join7_rdd = orders7_base_map.join(mapped7_customers_base)

In [57]:
join7_rdd.take(4)

[(4, (8827, 'CA')), (8, (2911, 'MA')), (12, (1837, 'TX')), (16, (7276, 'PR'))]

In [59]:
mapped7_rdd = join7_rdd.map(lambda x : (x[1][0],x[1][1]))

In [60]:
joinnew7_rdd = mapped7_rdd.join(order7_items_map)

In [61]:
joinnew7_rdd.take(5)

[(11586, ('LA', 59.99)),
 (11586, ('LA', 159.96)),
 (5760, ('FL', 199.92)),
 (5760, ('FL', 129.99)),
 (5760, ('FL', 199.98))]

In [62]:
reduced7_result = joinnew7_rdd.map(lambda x :  (x[1][0],x[1][1])).reduceByKey(lambda x,y:x+y).sortBy(lambda x : x[1], False)

In [64]:
reduced7_result.collect()

[('PR', 2304015.4199999785),
 ('CA', 998696.1400000022),
 ('NY', 398708.0900000002),
 ('TX', 337705.19000000035),
 ('IL', 260633.92000000016),
 ('FL', 163582.82000000007),
 ('OH', 131707.27999999997),
 ('PA', 130752.98999999999),
 ('MI', 124354.62),
 ('NJ', 109723.95999999993),
 ('AZ', 101481.85999999996),
 ('GA', 84056.73999999993),
 ('MD', 77687.17999999993),
 ('NC', 74614.50999999995),
 ('VA', 67826.67999999996),
 ('OR', 60647.269999999975),
 ('CO', 59099.15999999997),
 ('NV', 56741.21999999998),
 ('MA', 51603.72999999998),
 ('TN', 51442.92999999998),
 ('HI', 47428.46999999998),
 ('MO', 43604.09999999999),
 ('CT', 35892.119999999995),
 ('UT', 34088.079999999994),
 ('WI', 33611.44999999998),
 ('LA', 32678.14),
 ('WA', 32558.76999999999),
 ('NM', 31644.99999999999),
 ('MN', 20296.649999999998),
 ('SC', 19700.7),
 ('DC', 19391.3),
 ('KY', 16800.96),
 ('IN', 16133.990000000002),
 ('DE', 12988.520000000002),
 ('KS', 11386.759999999998),
 ('OK', 9883.939999999999),
 ('AR', 7839.36),
 ('WV

In [65]:
spark.stop()